# Actividad 8

### Analisis y entendimiento del Data Set

In [1]:
# empecemos importando las librerias necesarisas
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler, StringIndexer, Binarizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as MCE
from pyspark.ml.classification import DecisionTreeClassifier as DTC
from pyspark.ml.classification import GBTClassifier as GBTC
from pyspark.ml.classification import RandomForestClassifier as RFC

In [2]:
# Ceemos la sesion:
spark = SparkSession.builder.appName('IrisDS').getOrCreate()

21/09/24 16:50:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#leamos el data set
df = spark.read.csv('Iris_DataSheet.csv', header=True).cache()
# primer vistazo
df.show(10)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
+---+-------------+------------+-------------+------------+-----

In [4]:
# lineas del data set:
df.count()

150

In [5]:
# columnas y su tipo:
df.dtypes

[('Id', 'string'),
 ('SepalLengthCm', 'string'),
 ('SepalWidthCm', 'string'),
 ('PetalLengthCm', 'string'),
 ('PetalWidthCm', 'string'),
 ('Species', 'string')]

In [6]:
# Adecuemos el data set dando el fromato adecuado a las columnas numéricas
df = df.select(
    col('Id').cast('int'),
    col('SepalLengthCm').cast('float'),
    col('SepalWidthCm').cast('float'),
    col('PetalLengthCm').cast('float'),
    col('PetalWidthCm').cast('float'),
    col('Species')
)
df.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



## Transformar en vector los datos numéricos empleando Vector Assembler

In [7]:
# lo primero, definamos que datos(columnas) se han de combertir en vector:
req_feat = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']

# creemos los vectores
assembler = VectorAssembler(inputCols=req_feat, outputCol='Sizes')

# añadamoslo al dataset
df = assembler.transform(df)

In [8]:
df.show(10)

+---+-------------+------------+-------------+------------+-----------+--------------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|               Sizes|
+---+-------------+------------+-------------+------------+-----------+--------------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.09999990463256...|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.90000009536743...|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.69999980926513...|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.59999990463256...|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.5999999046...|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|[5.40000009536743...|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|[4.59999990463256...|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa

## Transformar los datos correspondientes a la columna especies usando StringIndexer

In [9]:
# convirtamos la columna 'Species' en una columnas numérica
df = StringIndexer(inputCol='Species', outputCol='SpeciesID').fit(df).transform(df)
# A setosa le asignará el 1, a versicolor el 2 y a virginica, el 3

In [10]:
df.show(10)

+---+-------------+------------+-------------+------------+-----------+--------------------+---------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|               Sizes|SpeciesID|
+---+-------------+------------+-------------+------------+-----------+--------------------+---------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.09999990463256...|      0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.90000009536743...|      0.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.69999980926513...|      0.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.59999990463256...|      0.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,3.5999999046...|      0.0|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|[5.40000009536743...|      0.0|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|[4

In [11]:
#quitemos la columna Species
df = df.drop('Species')

**Creemos ahora las particiones Train Test**

## Calcular la predicción y la precisión del modelo empleando Decision Tree Classifier DTC

In [12]:
#empelaremos el 85% del data set para Train y el 15% para test
(train, test) = df.randomSplit([0.85, 0.15])

In [13]:
# Antes de nada, creemos el modelo de prediccion
dtc = DTC(labelCol='SpeciesID', featuresCol='Sizes',maxDepth=10)

Aumentando el valor de 'maxDepth' de 5 a 10 la prediccion del modelo mejora, al menos en este caso, de ahi para arriba parece que se crea algo de overfitting

In [14]:
# Entrenemos el modelo:
dtc_model = dtc.fit(train)

In [15]:
# Generemos una prediccion:
dtc_pred = dtc_model.transform(test)
dtc_pred.toPandas().head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Sizes,SpeciesID,rawPrediction,probability,prediction
0,5,5.0,3.6,1.4,0.2,"[5.0, 3.5999999046325684, 1.399999976158142, 0...",0.0,"[45.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
1,24,5.1,3.3,1.7,0.5,"[5.099999904632568, 3.299999952316284, 1.70000...",0.0,"[45.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
2,28,5.2,3.5,1.5,0.2,"[5.199999809265137, 3.5, 1.5, 0.20000000298023...",0.0,"[45.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
3,37,5.5,3.5,1.3,0.2,"[5.5, 3.5, 1.2999999523162842, 0.2000000029802...",0.0,"[45.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
4,47,5.1,3.8,1.6,0.2,"[5.099999904632568, 3.799999952316284, 1.60000...",0.0,"[45.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0


### Evaluemos el modelo:

In [16]:
# configuremos el evaluador:
dtc_eval = MCE(labelCol='SpeciesID', predictionCol='prediction', metricName='accuracy')

In [17]:
#  Ahora si, evaluemos el modelo:
dtc_acc = dtc_eval.evaluate(dtc_pred)
print(f'DTC test accuracy: {dtc_acc}')

DTC test accuracy: 0.9523809523809523


## Calcular la predicción y la precisión del modelo empleando Gradient-boosted tree classifier GBTC

**Como el modelo GBTC solo permite trabajar con predicciones binarias, será necesario adecuar los datos:**<br/>
Para adecuar los datos haremos lo siguiente:
   * Primero, mediante la función Binarizer crearemos una columna llamada Virginica en la que se mostrara de froma bianria si un elemento es de la especie virginica o no.
   * Despues, empleando el mismo método, creareomo otra columna, Versicolor, en la que se mostrara si una elemento pertenece o no a las especies virginica o versicolor.
   * Por ultimo, haciendo uso de ambas columnas, estableceremos que elementos no pertenecen a la columna Versicolor

In [18]:
# Paso UNO
# creamos la nueva columna
vir = Binarizer(threshold=1.0, inputCol='SpeciesID', outputCol='Virginica')
# la añadimos a nuestro data frame
df = vir.transform(df)
#mostramos el resultado
df.show(150)

+---+-------------+------------+-------------+------------+--------------------+---------+---------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|               Sizes|SpeciesID|Virginica|
+---+-------------+------------+-------------+------------+--------------------+---------+---------+
|  1|          5.1|         3.5|          1.4|         0.2|[5.09999990463256...|      0.0|      0.0|
|  2|          4.9|         3.0|          1.4|         0.2|[4.90000009536743...|      0.0|      0.0|
|  3|          4.7|         3.2|          1.3|         0.2|[4.69999980926513...|      0.0|      0.0|
|  4|          4.6|         3.1|          1.5|         0.2|[4.59999990463256...|      0.0|      0.0|
|  5|          5.0|         3.6|          1.4|         0.2|[5.0,3.5999999046...|      0.0|      0.0|
|  6|          5.4|         3.9|          1.7|         0.4|[5.40000009536743...|      0.0|      0.0|
|  7|          4.6|         3.4|          1.4|         0.3|[4.59999990463256...|      0.0| 

In [19]:
# Paso DOS
# creamos la nueva columna
ver = Binarizer(threshold=0.0, inputCol='SpeciesID', outputCol='Versicolor')
# la añadimos a nuestro data frame
df = ver.transform(df)
#mostramos el resultado
df.show(150)

+---+-------------+------------+-------------+------------+--------------------+---------+---------+----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|               Sizes|SpeciesID|Virginica|Versicolor|
+---+-------------+------------+-------------+------------+--------------------+---------+---------+----------+
|  1|          5.1|         3.5|          1.4|         0.2|[5.09999990463256...|      0.0|      0.0|       0.0|
|  2|          4.9|         3.0|          1.4|         0.2|[4.90000009536743...|      0.0|      0.0|       0.0|
|  3|          4.7|         3.2|          1.3|         0.2|[4.69999980926513...|      0.0|      0.0|       0.0|
|  4|          4.6|         3.1|          1.5|         0.2|[4.59999990463256...|      0.0|      0.0|       0.0|
|  5|          5.0|         3.6|          1.4|         0.2|[5.0,3.5999999046...|      0.0|      0.0|       0.0|
|  6|          5.4|         3.9|          1.7|         0.4|[5.40000009536743...|      0.0|      0.0|    

In [20]:
# Paso Tres:
df = df.withColumn("Versicolor", when(df["Virginica"] == 1.0, 0.0).otherwise(df["Versicolor"]))

df.show(150)

+---+-------------+------------+-------------+------------+--------------------+---------+---------+----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|               Sizes|SpeciesID|Virginica|Versicolor|
+---+-------------+------------+-------------+------------+--------------------+---------+---------+----------+
|  1|          5.1|         3.5|          1.4|         0.2|[5.09999990463256...|      0.0|      0.0|       0.0|
|  2|          4.9|         3.0|          1.4|         0.2|[4.90000009536743...|      0.0|      0.0|       0.0|
|  3|          4.7|         3.2|          1.3|         0.2|[4.69999980926513...|      0.0|      0.0|       0.0|
|  4|          4.6|         3.1|          1.5|         0.2|[4.59999990463256...|      0.0|      0.0|       0.0|
|  5|          5.0|         3.6|          1.4|         0.2|[5.0,3.5999999046...|      0.0|      0.0|       0.0|
|  6|          5.4|         3.9|          1.7|         0.4|[5.40000009536743...|      0.0|      0.0|    

In [21]:
# Terminemos de adecuar el DF
df = df.select(
    col('SepalLengthCm'),
    col('SepalWidthCm'),
    col('PetalLengthCm'),
    col('PetalWidthCm'),
    col('Sizes'),
    col('Virginica').cast('float'),
    col('Versicolor').cast('float')
)
df.show(150)

+-------------+------------+-------------+------------+--------------------+---------+----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|               Sizes|Virginica|Versicolor|
+-------------+------------+-------------+------------+--------------------+---------+----------+
|          5.1|         3.5|          1.4|         0.2|[5.09999990463256...|      0.0|       0.0|
|          4.9|         3.0|          1.4|         0.2|[4.90000009536743...|      0.0|       0.0|
|          4.7|         3.2|          1.3|         0.2|[4.69999980926513...|      0.0|       0.0|
|          4.6|         3.1|          1.5|         0.2|[4.59999990463256...|      0.0|       0.0|
|          5.0|         3.6|          1.4|         0.2|[5.0,3.5999999046...|      0.0|       0.0|
|          5.4|         3.9|          1.7|         0.4|[5.40000009536743...|      0.0|       0.0|
|          4.6|         3.4|          1.4|         0.3|[4.59999990463256...|      0.0|       0.0|
|          5.0|     

In [22]:
df.dtypes

[('SepalLengthCm', 'float'),
 ('SepalWidthCm', 'float'),
 ('PetalLengthCm', 'float'),
 ('PetalWidthCm', 'float'),
 ('Sizes', 'vector'),
 ('Virginica', 'float'),
 ('Versicolor', 'float')]

Una vez hemos llegado a este punto, crearemos dos data Frames diferentes, uno para cada prediccion:

In [23]:
# obtengamos los nombre de las columnas que nos serán utiles mas adelante:
cols = df.columns

#creemos la primera versión del data set
sdf_1 = df

In [24]:
sdf_1.show(150)

+-------------+------------+-------------+------------+--------------------+---------+----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|               Sizes|Virginica|Versicolor|
+-------------+------------+-------------+------------+--------------------+---------+----------+
|          5.1|         3.5|          1.4|         0.2|[5.09999990463256...|      0.0|       0.0|
|          4.9|         3.0|          1.4|         0.2|[4.90000009536743...|      0.0|       0.0|
|          4.7|         3.2|          1.3|         0.2|[4.69999980926513...|      0.0|       0.0|
|          4.6|         3.1|          1.5|         0.2|[4.59999990463256...|      0.0|       0.0|
|          5.0|         3.6|          1.4|         0.2|[5.0,3.5999999046...|      0.0|       0.0|
|          5.4|         3.9|          1.7|         0.4|[5.40000009536743...|      0.0|       0.0|
|          4.6|         3.4|          1.4|         0.3|[4.59999990463256...|      0.0|       0.0|
|          5.0|     

In [25]:
#creemos un nuevo vector con sizes y speciesID_1.0 y con ello un nuevo data set
req = cols[-3:-2]
assembler = VectorAssembler(inputCols=req, outputCol='Sizes+')

# añadamoslo al dataset y eliminemos las colunas que sobran
sdf_2 = assembler.transform(df).drop('Sizes')

In [26]:
sdf_2.show(150) 

+-------------+------------+-------------+------------+---------+----------+--------------------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Virginica|Versicolor|              Sizes+|
+-------------+------------+-------------+------------+---------+----------+--------------------+
|          5.1|         3.5|          1.4|         0.2|      0.0|       0.0|[5.09999990463256...|
|          4.9|         3.0|          1.4|         0.2|      0.0|       0.0|[4.90000009536743...|
|          4.7|         3.2|          1.3|         0.2|      0.0|       0.0|[4.69999980926513...|
|          4.6|         3.1|          1.5|         0.2|      0.0|       0.0|[4.59999990463256...|
|          5.0|         3.6|          1.4|         0.2|      0.0|       0.0|[5.0,3.5999999046...|
|          5.4|         3.9|          1.7|         0.4|      0.0|       0.0|[5.40000009536743...|
|          4.6|         3.4|          1.4|         0.3|      0.0|       0.0|[4.59999990463256...|
|          5.0|     

**Empecemos con las predicciones de la primera especie:**

In [27]:
# primero crearemos la particion prescindiendo de la columna 'SpeciesID_2.0' para realizar ña primera predicción
(train_1, test_1) = sdf_1.randomSplit([0.85, 0.15])

# Segundo, crearemos la particion para la segunda prediccion:
(train_2, test_2) = sdf_2.randomSplit([0.85, 0.15])

In [28]:
# Creemos el modelo de la primera prediccion de la especie Virginica:
gbtc_1 = GBTC(labelCol='Virginica', featuresCol='Sizes', maxIter=10)

In [29]:
# Entrenemos el modelo de la predicción de Virginica:
gbtc_model_1 = gbtc_1.fit(train_1)

In [30]:
# una vez creado el modelo, hagamos la prediccion para la especie Virginica:
gbtc_pred_1 = gbtc_model_1.transform(test_1)
gbtc_pred_1.toPandas().head()

21/09/24 16:50:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/09/24 16:50:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Sizes,Virginica,Versicolor,rawPrediction,probability,prediction
0,4.9,3.0,1.4,0.2,"[4.900000095367432, 3.0, 1.399999976158142, 0....",0.0,0.0,"[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
1,5.0,3.2,1.2,0.2,"[5.0, 3.200000047683716, 1.2000000476837158, 0...",0.0,0.0,"[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
2,5.0,3.5,1.6,0.6,"[5.0, 3.5, 1.600000023841858, 0.6000000238418579]",0.0,0.0,"[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
3,5.5,2.4,3.8,1.1,"[5.5, 2.4000000953674316, 3.799999952316284, 1...",0.0,1.0,"[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
4,5.7,4.4,1.5,0.4,"[5.699999809265137, 4.400000095367432, 1.5, 0....",0.0,0.0,"[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0


Evaluemos la prediccion realizada:

In [31]:
# configuremos el evaluador:
gbtc_1_eval = MCE(labelCol='Virginica', predictionCol='prediction', metricName='accuracy')

In [32]:
#  Ahora si, evaluemos el modelo:
gbtc_1_acc = gbtc_1_eval.evaluate(gbtc_pred_1)
print(f'GBTC_1 test accuracy: {gbtc_1_acc}')

GBTC_1 test accuracy: 0.9285714285714286


Una vez tenemos la prediccion para Virginica, haremos para Versicolor, pero la haremos de un modo especial:

In [33]:
# adaptamos la prediccion obtenida a nuestras necesidades:
req = ['Sizes', 'prediction']
assembler = VectorAssembler(inputCols=req, outputCol='Sizes+')

#añadamoslo al dataset y eliminemos las colunas que sobran
gbtc_pred_1 = assembler.transform(gbtc_pred_1).drop()

#nos quedamos con lo que nos hace falta unicamente
gbtc_pred_1 = gbtc_pred_1.select(
    col('Virginica'),
    col('Versicolor'),
    col('Sizes+')
)
gbtc_pred_1.toPandas().head()

,Virginica,Versicolor,Sizes+
0,0.0,0.0,"[4.900000095367432, 3.0, 1.399999976158142, 0...."
1,0.0,0.0,"[5.0, 3.200000047683716, 1.2000000476837158, 0..."
2,0.0,0.0,"[5.0, 3.5, 1.600000023841858, 0.60000002384185..."
3,0.0,1.0,"[5.5, 2.4000000953674316, 3.799999952316284, 1..."
4,0.0,0.0,"[5.699999809265137, 4.400000095367432, 1.5, 0...."


In [34]:
#crearemos un nuevo modelo predictivo empleando columnas del segundo data set:
gbtc_2 = GBTC(labelCol='Versicolor', featuresCol='Sizes+', maxIter=10) 

In [35]:
# entrenaremos el modelo empleando test_2 obtenido de sdf_2:
gbtc_model_2 = gbtc_2.fit(train_2)

In [36]:
# Ahora en vez de realizar la prediccion sobre test_2, la realizaremos sobre gbtc_pred_1:
gbtc_pred_2 = gbtc_model_2.transform(gbtc_pred_1)
gbtc_pred_2.toPandas().head()

,Virginica,Versicolor,Sizes+,rawPrediction,probability,prediction
0,0.0,0.0,"[4.900000095367432, 3.0, 1.399999976158142, 0....","[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
1,0.0,0.0,"[5.0, 3.200000047683716, 1.2000000476837158, 0...","[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
2,0.0,0.0,"[5.0, 3.5, 1.600000023841858, 0.60000002384185...","[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0
3,0.0,1.0,"[5.5, 2.4000000953674316, 3.799999952316284, 1...","[-1.3259026792203317, 1.3259026792203317]","[0.06587782434721744, 0.9341221756527825]",1.0
4,0.0,0.0,"[5.699999809265137, 4.400000095367432, 1.5, 0....","[1.325902679220332, -1.325902679220332]","[0.9341221756527827, 0.06587782434721734]",0.0


In [37]:
# configuremos el evaluador nuevamente:
gbtc_2_eval = MCE(labelCol='Versicolor', predictionCol='prediction', metricName='accuracy')

In [38]:
#  evaluemos el modelo nuevamente:
gbtc_2_acc = gbtc_2_eval.evaluate(gbtc_pred_2)
print(f'GBTC_2 test accuracy: {gbtc_2_acc}')

GBTC_2 test accuracy: 1.0


**Sin embargo, haciendo la prediccion de esta forma la precision se ve afectada y la precion total de sistema sería:**

In [39]:
real_acc = gbtc_1_acc *gbtc_2_acc
print(f'Full prediction accuracy: {real_acc}')

Full prediction accuracy: 0.9285714285714286


## Calcular la predicción y la precisión del modelo empleando Random Forest Classifier RFC

In [40]:
# en esta caso es mas sencillo, primero, configuremos el modelo:
rfc = RFC(labelCol='SpeciesID', featuresCol='Sizes')

In [41]:
# Entrenemos el modelo con las mismas particiones creadas para el modelo DTC:
rfc_model = rfc.fit(train)

In [42]:
# realicemos la prediccion:
rfc_pred = rfc_model.transform(test)

In [43]:
# configuremos el evaluador:
rfc_eval = MCE(labelCol='SpeciesID', predictionCol='prediction', metricName='accuracy')

In [44]:
#  Ahora si, evaluemos el modelo:
rfc_acc = rfc_eval.evaluate(rfc_pred)
print(f'RFC test accuracy: {rfc_acc}')

RFC test accuracy: 0.9523809523809523


### Bibliografia:
   * Páfginas Web:
      * https://sparkbyexamples.com/pyspark/convert-pandas-to-pyspark-dataframe/
   * Otros:
       * Ejercicio 5 de la asignatura Fundamento de Big Data